# 1. Install Dependencies

In [1]:
!pip install transformers[torch]==4.3

  Using cached transformers-4.3.0-py3-none-any.whl (1.8 MB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
  creating build\lib.win-amd64-cpython-311\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
  creating build\lib.win-amd64-cpython-311\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
  creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-311\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31

In [2]:
!pip install pinecone-client

In [3]:
!pip install sentence-transformers torch

# 2. Load and Prepare Dataset

In [4]:
import json

def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

def prepare_dataset(train_file, val_file):
    train_data = load_data(train_file)
    val_data = load_data(val_file)

    train_dataset = []
    val_dataset = []

    for item_train in train_data:
        question_id_train = item_train['question_id']
        question_train = item_train['question']
        contexts_train = item_train['ctxs']
        answers_train = item_train['answers']

        train_dataset.append({
            'question_id': question_id_train,
            'question': question_train,
            'contexts': contexts_train,
            'answers': answers_train
        })

    for item_val in val_data:
        question_id_val = item_val['question_id']
        question_val = item_val['question']
        contexts_val = [ctx[0] for ctx in item_val['ctxs']]
        answers_val = item_val['answers']
        scores_val = [float(ctx[1]) for ctx in item_val['ctxs']]

        val_dataset.append({
            'question_id': question_id_val,
            'question': question_val,
            'contexts': contexts_val,
            'answers': answers_val,
            'scores': scores_val
        })

    return train_dataset, val_dataset

train_file = "D:/NLP Training Lab/ELI5.jsonl"
val_file = "D:/NLP Training Lab/ELI5_val.jsonl"

train_dataset, val_dataset = prepare_dataset(train_file, val_file)

In [5]:
import pandas as pd
train_dataset = pd.DataFrame(train_dataset)

train_dataset.head()

,question_id,question,contexts,answers
0,1oy5tc,in football whats the point of wasting the fir...,[46 defense Description To stop a passing game...,"[Keep the defense honest, get a feel for the p..."
1,2lojul,Why are different tiers (regular < mid < premi...,[have traditionally been three to four times t...,"[As someone who uses quality Premium, I wish t..."
2,8v5e3s,Stars and Visibility,"[of the intrinsic brightness of the sky, namel...",[It's a quirk of the human eye. At the center ...
3,1v3wij,How do we know all the money the government is...,[all future funds would be placed in selected ...,[I'm pretty confident most of it isn't going b...
4,2jlp6f,What are good and bad sides of manual and auto...,[amount of true manual control provided is hig...,"[Automatics weigh more, so that alone makes ga..."


In [6]:
import numpy as np
train_dataset.shape

(272634, 4)

In [7]:
train_dataset = train_dataset.drop("question_id", axis=1)

In [8]:
train_dataset.head()

,question,contexts,answers
0,in football whats the point of wasting the fir...,[46 defense Description To stop a passing game...,"[Keep the defense honest, get a feel for the p..."
1,Why are different tiers (regular < mid < premi...,[have traditionally been three to four times t...,"[As someone who uses quality Premium, I wish t..."
2,Stars and Visibility,"[of the intrinsic brightness of the sky, namel...",[It's a quirk of the human eye. At the center ...
3,How do we know all the money the government is...,[all future funds would be placed in selected ...,[I'm pretty confident most of it isn't going b...
4,What are good and bad sides of manual and auto...,[amount of true manual control provided is hig...,"[Automatics weigh more, so that alone makes ga..."


In [28]:
train_dataset = train_dataset[:1000]

# 3. Pinecone

In [29]:
import pinecone

pinecone.init(
    api_key="c7b42c76-1aa6-4990-8427-4192df69b5d1",
    environment="asia-southeast1-gcp-free"
)

In [30]:
index_name = "abstractive-question-answering"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

index = pinecone.Index(index_name)

# 4. Retriever

In [31]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU (if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# 5. Embeddings and Upsert

In [32]:
!pip install tqdm

In [33]:
from tqdm import tqdm
batch_size = 64

for i in tqdm(range(0, len(train_dataset), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(train_dataset))
    # extract batch
    batch = train_dataset.iloc[i:i_end]
    # generate embeddings for batch
    emb = []
    for context in batch["contexts"]:
        emb_batch = retriever.encode(context).tolist()
        emb.extend(emb_batch)
    # metadata
    meta = batch[["question", "contexts", "answers"]].to_dict(orient="records")
    # IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

index.describe_index_stats()


100%|██████████| 2/2 [08:47<00:00, 263.67s/it]


{'dimension': 768,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 8384}},
 'total_vector_count': 8384}

# 6. Generator

In [34]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

In [35]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    return xc

In [38]:
def format_query(query, context):
    # extract contexts
    context = [f"<P> {m['metadata']['contexts']}" for m in context]
    # concatinate all contexts
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [39]:
query = "How do muscles grow?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': '624',
              'metadata': {'contexts': ['or statements are considered hate '
                                        'speech or inappropriate ethnic slurs '
                                        '(such as using the word Hun to a '
                                        'German, using the word Jap to a '
                                        'Japanese person, etc.). In most '
                                        'modern cultures, insulting a person '
                                        'or group of people, especially for '
                                        'any reason outside their immediate '
                                        'control, such as having a medical '
                                        'condition, following a particular '
                                        'religion, or being poor, is '
                                        'considered rude. Rude speech also '
                                        'includes der